## Gathering Dependencies

_Importing Required Libraries_

In [1]:
pip install hampel

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Anurag Dutta\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
from numpy import array

## Pretraining

_Van der Pol intermittancy_

The `van_der_pol_intermittency.dat` is generated from the Runge Kutta Method.

This feeds the model with the dynamics of the Van der Pol Oscillator

In [3]:
data = np.genfromtxt('datasets/van_der_pol_intermittency.dat')
training_set = pd.DataFrame(data).reset_index(drop=True)
training_set = training_set.iloc[:,1]

## Computing the Gradients

_Calculating the value of_ $\frac{dx}{dt}$, _and_ $\frac{d^2x}{dt^2}$

In [4]:
t_diff = 1 # Daily Data
print(training_set.max())
gradient_t = (training_set.diff()/t_diff).iloc[1:]
print(gradient_t)
gradient_tt = (gradient_t.diff()/t_diff).iloc[1:]
print(gradient_tt)

1.0
1        -0.000100
2        -0.000204
3        -0.000312
4        -0.000425
5        -0.000541
            ...   
139867    0.004633
139868    0.004629
139869    0.004624
139870    0.004619
139871    0.004614
Name: 1, Length: 139871, dtype: float64
2        -0.000104
3        -0.000108
4        -0.000112
5        -0.000117
6        -0.000121
            ...   
139867   -0.000005
139868   -0.000005
139869   -0.000005
139870   -0.000005
139871   -0.000005
Name: 1, Length: 139870, dtype: float64


## Loading Datasets

_दिल्ली WIND SPEED_

In [5]:
data = pd.read_csv("datasets/delhi.csv")
training_set = data.iloc[:, 3]
print(training_set)

0       0.000000
1       2.980000
2       4.633333
3       1.233333
4       3.700000
          ...   
1457    3.547826
1458    6.000000
1459    6.266667
1460    7.325000
1461    0.000000
Name: wind_speed, Length: 1462, dtype: float64


In [6]:
training_set = training_set.reset_index(drop=True)
gradient_t = gradient_t.reset_index(drop=True)
gradient_tt = gradient_tt.reset_index(drop=True)
print(gradient_t)
print(gradient_tt)

0        -0.000100
1        -0.000204
2        -0.000312
3        -0.000425
4        -0.000541
            ...   
139866    0.004633
139867    0.004629
139868    0.004624
139869    0.004619
139870    0.004614
Name: 1, Length: 139871, dtype: float64
0        -0.000104
1        -0.000108
2        -0.000112
3        -0.000117
4        -0.000121
            ...   
139865   -0.000005
139866   -0.000005
139867   -0.000005
139868   -0.000005
139869   -0.000005
Name: 1, Length: 139870, dtype: float64


In [7]:
print(gradient_t.shape)
print(training_set.shape[:-1])
df = pd.concat((training_set[:-1], gradient_t), axis=1)
gradient_tt.columns = ["grad_tt"]
df = pd.concat((df[:-1], gradient_tt), axis=1)
df.columns = ['y_t', 'grad_t', 'grad_tt']

(139871,)
()


## Preprocessing the data into supervised learning

In [8]:
def Supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
      cols.append(df.shift(-i))
      if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
      else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
       agg.dropna(inplace=True)
    return agg    

In [9]:
dat = Supervised(df.values, n_in = 10, n_out = 365)
dat.columns

Index(['var1(t-10)', 'var2(t-10)', 'var3(t-10)', 'var1(t-9)', 'var2(t-9)',
       'var3(t-9)', 'var1(t-8)', 'var2(t-8)', 'var3(t-8)', 'var1(t-7)',
       ...
       'var3(t+361)', 'var1(t+362)', 'var2(t+362)', 'var3(t+362)',
       'var1(t+363)', 'var2(t+363)', 'var3(t+363)', 'var1(t+364)',
       'var2(t+364)', 'var3(t+364)'],
      dtype='object', length=1125)

In [10]:
data = Supervised(df.values, n_in = 10, n_out = 365)
data.drop(['var2(t-10)', 'var3(t-10)', 'var2(t-9)', 'var3(t-9)', 'var2(t-8)',
       'var3(t-8)', 'var2(t-7)', 'var3(t-7)', 'var2(t-6)', 'var3(t-6)',
       'var2(t-5)', 'var3(t-5)', 'var2(t-4)', 'var3(t-4)', 'var2(t-2)',
       'var3(t-2)', 'var2(t-1)', 'var3(t-1)','var2(t-3)', 'var3(t-3)'], axis = 1, inplace = True)#,18,19
print(data.head())
print(data.columns)

    var1(t-10)  var1(t-9)  var1(t-8)  var1(t-7)  var1(t-6)  var1(t-5)  \
10    0.000000   2.980000   4.633333   1.233333   3.700000   1.480000   
11    2.980000   4.633333   1.233333   3.700000   1.480000   6.300000   
12    4.633333   1.233333   3.700000   1.480000   6.300000   7.142857   
13    1.233333   3.700000   1.480000   6.300000   7.142857  12.500000   
14    3.700000   1.480000   6.300000   7.142857  12.500000   7.400000   

    var1(t-4)  var1(t-3)  var1(t-2)  var1(t-1)  ...  var3(t+361)  var1(t+362)  \
10   6.300000   7.142857  12.500000   7.400000  ...     0.000008     1.850000   
11   7.142857  12.500000   7.400000  10.571429  ...     0.000008     1.233333   
12  12.500000   7.400000  10.571429  13.228571  ...     0.000008     0.000000   
13   7.400000  10.571429  13.228571   4.633333  ...     0.000008     6.725000   
14  10.571429  13.228571   4.633333   0.616667  ...     0.000008     3.250000   

    var2(t+362)  var3(t+362)  var1(t+363)  var2(t+363)  var3(t+363)  \
10 

## Train and Test Split

In [11]:
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train_1 = np.array(data[0:train_size])
test_1 = np.array(data[train_size:len(data)])
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train_1)
test = scaler.transform(test_1)
trainY = train[:,-9:]
trainX = train[:,:-9]
testY = test[:,-9:]
testX = test[:,:-9]
trainX = trainX.reshape((trainX.shape[0], 1, trainX.shape[1]))
testX = testX.reshape((testX.shape[0], 1, testX.shape[1]))
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

(869, 1, 1096) (869, 9) (218, 1, 1096) (218, 9)


## Model

In [12]:
model = Sequential()
model.add(LSTM(50, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(9))
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(trainX, trainY, epochs=500, batch_size=64, validation_data=(testX, testY), shuffle=False)

Epoch 1/500
14/14 [==============================] - 2s 41ms/step - loss: 0.1165 - val_loss: 0.1025
Epoch 2/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0776 - val_loss: 0.0708
Epoch 3/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0459 - val_loss: 0.0367
Epoch 4/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0362 - val_loss: 0.0274
Epoch 5/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0269 - val_loss: 0.0227
Epoch 6/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0251 - val_loss: 0.0202
Epoch 7/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0258 - val_loss: 0.0142
Epoch 8/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0195 - val_loss: 0.0140
Epoch 9/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0177 - val_loss: 0.0107
Epoch 10/500
14/14 [==============================] - 0s 5ms/step - loss: 0.0164 - val_loss: 0.0133
Epoch 11

## Metrics

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
yhat = model.predict(testX)
print(yhat.shape)
testX = testX.reshape((testX.shape[0], testX.shape[2]))
print(testX.shape)
inv_yhat = np.concatenate((testX, yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat1 = inv_yhat[:, -3:]
inv_yhat = inv_yhat[:, -3]
inv_y = np.concatenate((testX, testY), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y1 = inv_y[:, -3:]
inv_y = inv_y[:, -3]
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

7/7 [==============================] - 0s 2ms/step
(218, 9)
(218, 1096)
Test RMSE: 4.867
Test MAE: 3.776
